In [ ]:
%matplotlib inline
from IPython.display import HTML

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [ ]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

# Table of content <a class="anchor" id="tablecontent"></a>
## [1. Introduction](#intro)
## [2. Method](#method)
## [3. Additional data](#add-data)
## [4. Localisation of borrowers](#localisation)
### [4.1 Localisation of borrowers by country](#localisation-country)
#### [4.1.1 Annual mean number of loans](#country-annual-mean)
#### [4.1.2 Country's population](#country-population)
#### [4.1.3 Number of loans per 10000 inhabitants](#country-loans-vs-pop)
#### [4.1.4 Annual mean loan amount](#country-mean-loan)
#### [4.1.5 Annual mean loan amount / GDP per capita](#country-mean-loan-vs-gdp)
#### [4.1.6 Correlation analysis](#country-correlation)
### [4.2 Localisation of borrowers by sector](#localisation-sector)
#### [4.2.1 Loans distribution by sector on the global scale](#sector-wordwide)
#### [4.2.2 Loans distribution by sector and country ](#sector-country)
### [4.3 Localisation of borrowers by gender](#gender)
#### [4.3.1 Loans distribution by gender on the global scale](#gender-ww)
#### [4.3.2 Loans distribution by gender and country ](#gender-nat)
## [5. Welfare assessment](#welfare-assessment)
### [5.1 Worldwide assessment](#ww-assessment)
#### [5.1.1 Categorical variables](#ww-assessment-cat)
#### [5.1.2 Loan activity clustering](#ww-assessment-loan-uses-clustering)
#### [5.1.3 Loan activity cluster prediction for new borrowers](#ww-assessment-loan-uses-prediction)
#### [5.1.4 Welfare assessment of borrowers in the database](#ww-assessment-borrowers)
#### [5.1.5 Welfare assessment of new borrowers](#ww-assessment-new-borrowers)
### [5.2 Nationwide assessment](#nw-assessment)
### [5.3 Welfare assessment by country and sector](#ns-assessment)
## [6. Conclustions](#conclusions)

# 1. Introduction <a class="anchor" id="intro"></a>

Kiva has provided a dataset of loans issued over the last two years, and participants are invited to use this data as well as source external public datasets to help Kiva build models for assessing borrower welfare levels.  With a stronger understanding of their borrowers and their poverty levels, Kiva will be able to better assess and maximize the impact of their work.

The present submission aims at responding to the queries regarding the following criteria:

1. __Localization__ - The highly localized borrower situation will be highlighted

2. __Execution__ - Python functions will be built so that Kiva’s team can readily employ them in their impact calculations.

3. __Ingenuity__ -  The submission aims at providing a model for assessing welfare levels of borrowers, both on a global scale and nation-, sector-wise.

# 2. Method <a class="anchor" id="method"></a>

To assess the welfare levels of borrowers,  quantitative statistical methods will be applied to data from Kiva coupled with worldwilde data on economic development and poverty levels provided by World Bank.

It is worth emphasizing that to ensure a meaningful interpretation of the data, in particular for the purpose of inter-countries comparaison, we will consider the different measures by their:
-  absolute values

and
-  __relative values__

The absolute and relative values are both important, but in different ways and to different people. 

For those who work at Kiva and worldwide lenders, the absolute values tend to be the most important measures to monitor. These measures allow one to know, for instance, how many loans and what is total the amount of loans in a particular country or globally.

However, to evaluate the potential effects of loans to borrowers or assess welfare levels of borrowers from different countries, the relative values represent more meaningful measures. For instance, a  200 USD (absolute value) loan means significantly differently to an Afghan and an American, which is due to the different living standard (income and expense levels) in the respective countries. The ratio of the loan amount to the mean income level (a relative measure) might help revealing the different effects of loans to borrowers. Similarly, there are many loans in Philippines, of which the country's population is also important. In order to compare numbers of loans in countries, a relative measure (e.g. #loans/population size) might be used.

Therefore, in the submission, we are considering both absolute and relative measures. The reader will see that for investigating the __localisation of loans__, the absolute and relative measures sometimes reveal different facts. 


To __assess the welfare levels of borrowers__, it is of utmost importance to find out the most important features of borrowers, then associate these features to determined welfare classes. This is indeed a problem of __clustering__ (divide the inial borrowers sample into distinctive clusters), then __classification__ (given new borrowers with loan details, determine which welfare class they belong to). 

For this purpose, the features of borrowers need to be identified. For numerical features (e.g. loan amount), no further pre-processing is required. Otherwise, categorical variables are used to describe the features (e.g. gender, sector). To this end, the function LabelEncoder of sklean was utilized. In particular, for the columns loan use and borrower activity, __natural language processing__ techniques were used to detect the most important features (package __nltk__). The raw text was numericalized in the form of a tf_idf (term frequency-inverse document frequency) matrix. tf_idf matrix is used to weigh how important a word of a document in a document collection. tf-idf weight for a term is the product of its tf weight and idf weight. The most relevant terms will be retrieved by using __k-means__ clustering technique on the tf_idf matrix.

Once the loans have been clustered, new loans can be easily assigned to an existing cluster (nltk object is used again to convert raw text to numerical values (e.g. Section 5.1.3), then k-means object is utilized to classify the loans (e.g. Section 5.1.5) )

# 3. Additional data <a class="anchor" id="add-data"></a>

Data provided by Kiva will be coupled with external data. More precisely, we collected from the __World Bank__ data on:
- national demographic data (e.g. population size), 
- economic situations (e.g. annual GDP, annual income), 
- poverty levels (e.g. MPI, rate of population living in poverty) 

The data from __World Bank__ was collected using the package __pandas_datareader__ [documentation](https://pandas-datareader.readthedocs.io/en/latest/remote_data.html#remote-data-wb). This package allows easy search for interested measures. For instance,  the following commands:

from pandas_datareader import wb
wb.search(string = 'Development', field = 'name')

will return the measures containing the term Development in their name field. In deed, use of pandas_datareader on kaggle platform failed due to certain connection problems. For the sake of simplicity, we collected the data then uploaded them as a data source on kaggle.

In [ ]:
import sys  
#reload(sys)  
#sys.setdefaultencoding('utf8')

import pandas as pd
#from pandas_datareader import wb
pd.set_option("display.max_colwidth",200)

import matplotlib.pyplot as plt
import seaborn as sns

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import plotly.figure_factory as ff

import numpy as np

import nltk
#nltk.download('popular')
import pandas as pd

import pickle

from bs4 import BeautifulSoup
import re
#import os
#import codecs
from sklearn import feature_extraction
#import mpld3
from nltk.stem.snowball import SnowballStemmer

from sklearn import preprocessing

from sklearn.cluster import KMeans

from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
#%% define function to plot world maps from series

def plot_world_map_from_series(df, filename = None):
    
    df = df.reset_index()
    # by default, first columns is country, second columns is value
    
    data = [ dict(
        type = 'choropleth',
        locations = df.ix[:,0],
        locationmode = 'country names',
        z = df.ix[:,1].astype('float'),
#        text = df.ix[:,0].str.cat( df.ix[:,1].astype('str'), sep = ' '),
        text = df.ix[:,0], 
#        colorscale = 'Blues',
        autocolorscale = True,
        reversescale = False,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            ) ),
        colorbar = dict(
            tickprefix = '',
            #title = df.columns[1]
        )
      ) ]

    layout = dict(
        title = df.columns[1],
        geo = dict(
            showframe = False,
            showcoastlines = False,
            projection = dict(
                type = 'Mercator'
            )
        )
    )

    if filename == None:
        filename = df.columns[1]

    fig = dict( data=data, layout=layout )
    return py.iplot( fig, validate = True, filename = filename)
    
#%% plot horizontal bars from series
def plot_barh_from_series(df, filename = None):
    
    df = df.reset_index()
    # by default, first columns is country, second columns is value
    
    trace = go.Bar(
        y= df.ix[:,0],
        x=df.ix[:,1],
        orientation = 'h',
        marker=dict(
            color=df.ix[:,1],
            autocolorscale = True,
            reversescale = False
        ),
    )
    
    layout = go.Layout(
        title= df.columns[1],
        width=800,
        height=1200,
        )
    data = [trace]
    
    fig = go.Figure(data=data, layout=layout)
    
    if filename == None:
        filename = df.columns[1]
    
    return py.iplot(fig, filename= filename)

#%% plot correlation matrix from pandas frame
def plot_correlation_matrix(corr, xcols = None, ycols = None, filename = None, title = None):
    # corr is the correlation matrix obtained from a dataframe using pandas
    
    if xcols == None:
        xcols = corr.columns.tolist()
    if ycols == None:
        ycols = corr.columns.tolist()
    
    layout = dict(
        title = title,
        width = 800,
        height = 800,
#        margin=go.Margin(l=100, r=10, b=50, t=50, pad=5),
        margin=go.Margin(l=250, r=50, b=50, t=250, pad=4),
        yaxis= dict(tickangle=-30,
                    side = 'left',
                    ),
        xaxis= dict(tickangle=-30,
                    side = 'top',
                    ),
    )
    fig = ff.create_annotated_heatmap(
        z=corr.values,
        x= xcols,
        y= ycols,
        colorscale='Portland',
        reversescale=True,
        showscale=True,
        font_colors = ['#efecee', '#3c3636'])
    fig['layout'].update(layout)
    
    if filename == None:
        filename = 'correlation matrix'
    return py.iplot(fig, filename= filename)

In [ ]:
#%% load data from kiva
data_kvloans = pd.read_csv("../input/data-science-for-good-kiva-crowdfunding/kiva_loans.csv")

In [ ]:
# using mpi from OPHI. it has been shown that mpi data from kiva is prone to inaccuracy, in particular in GPS locations
# https://www.kaggle.com/marcomarchetti/ophi-subnat-decomp-headcount-mpi-201718/data
data_mpi = pd.read_csv('../input/ophi-subnat-decomp-headcount-mpi-201718/OPHI_SubNational_Decomposition_HeadCount_MPI_2017-18.csv', index_col = None, nrows =993, encoding = 'latin1', usecols = ['ISO_Country_Code', 'Country', 'Sub_national_region', 'World_region', 'Survey', 'Year', 'Population_Share', 'Country_MPI', 'Region_MPI', 'Schooling', 'Child_school_attendance', 'Child_mortality ', 'Nutrition', 'Electricity', 'Improved_sanitation', 'Drinking_water', 'Floor', 'Cooking _fuel', 'Asset_ownership', 'Num_of_indic', 'Indic_missing'])

In [ ]:
data_kvmpi = pd.read_csv("../input/data-science-for-good-kiva-crowdfunding/kiva_mpi_region_locations.csv")
data_kvmpi.dropna(axis= 0, thresh = 2, inplace = True)

In [ ]:
# column of year when loan funded
data_kvloans['year']  = pd.to_datetime(data_kvloans['date']).dt.year.astype(str)

# change name of Cote d'Ivoire
data_kvloans['country'] = data_kvloans['country'].str.replace("Cote D'Ivoire","Cote d'Ivoire")

In [ ]:
data_wb = pd.read_csv('../input/datawb/data_wb.csv')
data_wb['year'] = data_wb['year'].astype(str)
data_wb.set_index(['country','year'], inplace = True)

In [ ]:
# data poverty
data_poverty = pd.read_csv('../input/datawb/data_poverty.csv')
data_poverty['year'] = data_poverty['year'].astype(str)
data_poverty.set_index(['country','year'], inplace = True)

In [ ]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

# 4. Localisation of borrowers <a class="anchor" id="localisation"></a>
[Click here to go to Table of contents](#tablecontent)

As proposed, we compute relative measures when data is available. For instance, knowing the annual total number of loans and the total population of each country, one can calculate the number of Kiva loans per 10000 inhabitants in the country. Similarly, the annual mean loan amount as percentage of annual GDP per capita is obtained.

In [ ]:
#%% put all necessary data into a single frame
df1 = data_kvloans.groupby(by = ['country', 'year'])['loan_amount'].mean()
df1.name = 'Mean loan amount'

df2 = data_kvloans.groupby(by = ['country', 'year'])['loan_amount'].sum()
df2.name = 'Total loan amount'

# number of loans in each country in each years
df3 = data_kvloans.groupby(by = ['country', 'year'])['loan_amount'].size()
df3.name = '# loans'

df = pd.concat( [data_wb, df1, df2, df3 ], axis = 1, join = 'outer' )

df['# loans in 10000 inhabitants'] = 10000.*df['# loans'] /df['Total population']

df['Mean loan amount / GDP per capita (current US$) (%)'] = 100.* df['Mean loan amount'] / df['GDP per capita (current US$)']


First, we depict in the following figure the Pearson correlation coefficients calculated for each couple of investigated measures. 

The annual mean loan amount increases with increasing GDP per capita (corr. coeef. = +0.26), which means that people in countries with higher GDP per capita obtain loans at higher average amount. Meanwhile, the ratio mean loan amount/GDP per capita decreases with increasing GDP per capita, which means the loan amount represents a smaller portion.

It is also worth emphasizing the slight decrease of the number of loans (both #loans and #loans in 10000 inhabitants) when the following measures increase:
- GDP/capita, 
- mean loan amount,
- mean loan amount/GDP per capita.


In [ ]:
plot_correlation_matrix( df.corr().round(2))

In [ ]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

# 4.1 Localisation of borrowers by country <a class="anchor" id="localisation-country"></a>
[Click here to go to Table of contents](#tablecontent)

## 4.1.1 Annual mean number of loans <a class="anchor" id="country-annual-mean"></a>
[Click here to go to Table of contents](#tablecontent)

One first considers the annual mean number of loans in each country.

In [ ]:
dfaux = df.reset_index().groupby('country')['# loans'].mean()
dfaux = dfaux.fillna(value = 0)
plot_barh_from_series( dfaux.sort_values(ascending = False).head(40) )

l1 = dfaux.sort_values(ascending = False).index.tolist()

Hereafter one finds the list of 5 countries with the highest annual number of loans in the descending order

In [ ]:
df.ix[ l1[:5]]

And the number of loans per year represented on a world map. Philippines and Kenya are two countries with outstanding annual mean number of loans.

In [ ]:
plot_world_map_from_series( dfaux )

In [ ]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

## 4.1.2 Total population <a class="anchor" id="country-population"></a>
[Click here to go to Table of contents](#tablecontent)

However, it is worth noting the differences in countries populations. One finds below the population size in the ascending order of annual mean number of loans.

In [ ]:
dfaux = df.reset_index().groupby('country')['Total population'].mean().loc[l1]

plot_barh_from_series( dfaux.head(30) )

## 4.1.3 Number of loans per 10000 inhabitants<a class="anchor" id="country-loans-vs-pop"></a>
[Click here to go to Table of contents](#tablecontent)

Herein we consider the number of loans per 10000 inhabitants, which allow to take into account the effect of countries populations on the absolute number of loans. The following figure depicts this measure in the ascending order of absolute number of loans.

In [ ]:
dfaux = df.reset_index().groupby('country')['# loans in 10000 inhabitants'].mean().loc[l1]

plot_barh_from_series( dfaux.head(30) )

The fact is that when considering the number of loans per 10000 inhabitants, it is neither Philippines nor Kenya which takes the first positions. It is Samoa, then El Salvador, then Tajikistan, Cambodia, Nicaragua.

We are plotting the number of loans per 10000 inhabitants on a world map without Samoa (which is an outlier in the middle of the Pacific ocean) and El Salvador.

A localisation of loans is observed in South America, South East Asia and certain parts of Africa

In [ ]:
dfaux = df.reset_index().groupby('country')['# loans in 10000 inhabitants'].mean()
# remove Samoa (too different)
dfaux = dfaux[ ~dfaux.index.isin([ 'Samoa', 'El Salvador'] ) ]
# replace nan with 0 (no loan)
dfaux = dfaux.fillna(value = 0)
plot_world_map_from_series( dfaux )

The following table contains details about the Samoa and El Salvador

In [ ]:
df.ix[ ['Samoa', 'El Salvador']]

## 4.1.4 Number of loans per 10000 poor inhabitants<a class="anchor" id="country-loans-vs-poor-pop"></a>
[Click here to go to Table of contents](#tablecontent)

We considered the ratio between the number of loans and population size. However, it is even more appealing to investigate the number of loans over the size of poor population in each country.

In [ ]:
# join mean values of two dataframes (over different periods, assumption that the trend stays the same for poverty data)
df_pov = pd.concat([df.reset_index().groupby('country').mean(), data_poverty.reset_index().groupby('country').mean(), data_kvmpi.groupby('country')['MPI'].mean() ], axis = 1, join = 'outer')

In [ ]:
df_pov[u'# loans in 10000  poor inhabitants'] = 10000. * df_pov[u'# loans'] / (df_pov[u'Total population'] * df_pov[u'Poverty headcount ratio at national poverty lines (% of population)'] / 100.)

In [ ]:
plot_barh_from_series( df_pov.loc[l1[:30],u'# loans in 10000  poor inhabitants'])

There was no major difference compared to the number of loans per 10000 inhabitants in the previous figure, except the fact that for some countries, no data was available regarding the size of poor population (e.g. Kenya)

In [ ]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

## 4.1.5 Annual mean loan mount <a class="anchor" id="country-mean-loan"></a>
[Click here to go to Table of contents](#tablecontent)

We considered the question: how many people in each country received the loans? The next question to answer is: how much did the receive?

We first calculate the annual mean loan amount in each country, the plot this quantity in the ascending order of annual mean number of loans

In [ ]:
dfaux = df.reset_index().groupby('country')['Mean loan amount'].mean()
dfaux = dfaux.fillna(value = 0.)
dfaux.drop(['Virgin Islands (U.S.)'], inplace = True)

In [ ]:
plot_barh_from_series( dfaux.ix[l1[:30] ] )

The figure illustrates the fact that the number of loans and annual mean loan amount is negatively correlated. In terms of mean loan amount, it is neither Philippines nor Kenya, El Salvador that is at the top of the list. It is the USA. The mean loan amount is plotted on a world map (without Cote d'Ivoire, which is an exceptionnal outlier)

In [ ]:
plot_world_map_from_series( dfaux.loc[ ~(dfaux.index == "Cote d'Ivoire")]) # remove Cote d'Ivoire

Indeed, there was only one loan in Cote d'Ivoire worth 50000 USD, which allowed the creation of new jobs for 200 workers.

In [ ]:
data_kvloans.loc[  data_kvloans['country'] == "Cote d'Ivoire"]

In [ ]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

## 4.1.6 Annual mean loan mount / GDP per capita <a class="anchor" id="country-mean-loan-vs-gdp"></a>
[Click here to go to Table of contents](#tablecontent)

It is however important to emphasize that the same loan amount does not mean the same thing to borrowers in different countries. For instance, a 500$ loan might be a forturne for a poor Afghan, whereas the same amount does not represent a huge amount for an American. The income varies from one country to another, from one region to another. Therefore, we propose the use of the ratio (in %): annual mean loan amount / annual GDP per capita. This measure takes into account the national wealth (in terms of GDP per capita), thus allows a more reasonable comparaison between countries.

In [ ]:
dfaux = df.reset_index().groupby('country')['Mean loan amount / GDP per capita (current US$) (%)'].mean().loc[l1]
dfaux = dfaux.fillna(value = 0)

plot_barh_from_series( dfaux.ix[l1[:70]] , filename = 'mean loan vs. gdp per capita barh')

A world map plot of the ratio mean loan amount / GDP per capita. 

In [ ]:
plot_world_map_from_series( dfaux, filename = 'mean loan vs. gdp per capita map')

This allows to identify a strong localisation in Africa when considering the ratio mean loan amount vs. GDP per capita. It happens that even a small loan amount (in absolute values) in those countries (Burundi, Somalia, Afganistan, Malawi, etc.) represents a 1000% of the country's annual GDP per capita.

## 4.1.7 Correlation analysis <a class="anchor" id="country-correlation"></a>
[Click here to go to Table of contents](#tablecontent)

We enrich our analysis with data indicating the poverty levels of countries, namely the poverty gaps at different levels, the MPI, income share held by the poorest population.

In [ ]:
xcols = ['Total population','GDP per capita (current US$)',u'Income share held by lowest 10%',u'Income share held by lowest 20%',u'Poverty gap at $1.90 a day (2011 PPP) (%)',u'Poverty gap at national poverty lines (%)',u'Poverty gap at $5.50 a day (2011 PPP) (% of population)',u'Poverty headcount ratio at $1.90 a day (2011 PPP) (% of population)',u'Poverty headcount ratio at $3.20 a day (2011 PPP) (% of population)',u'Poverty headcount ratio at $5.50 a day (2011 PPP) (% of population)',u'Poverty headcount ratio at national poverty lines (% of population)','MPI']
ycols = ['Total loan amount', '# loans','# loans in 10000 inhabitants', '# loans in 10000  poor inhabitants', 'Mean loan amount', 'Mean loan amount / GDP per capita (current US$) (%)']
plot_correlation_matrix( df_pov.corr().loc[ycols, xcols].round(2), xcols = xcols, ycols = ycols, filename = 'correlation matrix poverty' )

The above analysis of correlation allows to enrich the previous obsevations. The above figure depicts the linear correlation coefficient between all measures considered previously (including those derived from given data, and those combined with external sources). The following facts are observed:

The poorer the population is, the smaller the total number (as well as its ratio in 10000 poor inhabitants) and total amount of KIVA loans  become. 

However, for those in poor countries who receive KIVA loans, the mean values of loans is positively correlated with the poverty level, thus these loans represent a higher percentage with respect to the country GDP per capita.

As a summary, people at lower welfare levels get less KIVA loans (in total number of loans, in total amount of loans). These loans represent an important ratio with respect to their average income.

In [ ]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

# 4.2 Localisation of borrowers by sector <a class="anchor" id="localisation-sector"></a>
[Click here to go to Table of contents](#tablecontent)

In [ ]:
dfkv = data_kvloans[ [u'country', u'year', u'activity', u'sector', u'loan_amount']]

## 4.2.1 Loans distribution by sector on the global scale <a class="anchor" id="sector-wordwide"></a>

We now investigate the distribution of loans by sector on the worlwide level.

First, the number of loans by sector is depicted in the following pie chart. Hover over the figure to see the absolute annual number of loans.

In [ ]:
trace = go.Pie(sort = False,labels=dfkv.groupby('sector').size().index.tolist(), values=list(dfkv.groupby('sector').size().values))
fig = {
    'data': [trace],
    'layout': {'title': '# loans by sector'}
     }

py.iplot(fig)

Below we find the total loan amount by sector:

In [ ]:
trace = go.Pie(sort = False,labels=dfkv.groupby('sector')['loan_amount'].sum().index.tolist(), values=list(dfkv.groupby('sector')['loan_amount'].sum().values))
fig = {
    'data': [trace],
    'layout': {'title': 'Total loan amount by sector'}
     }

py.iplot(fig)

The mean loan amount by sector is distributed as follows:

In [ ]:
trace = go.Pie(sort = False, labels=dfkv.groupby('sector')['loan_amount'].mean().index.tolist(), values=list(dfkv.groupby('sector')['loan_amount'].mean().values))
fig = {
    'data': [trace],
    'layout': {'title': 'Mean loan amount by sector'}
     }

py.iplot(fig)

We can draw the following observations: 

- The total loan amount in sectors of agriculture, food and retail is significantly higher than in other sectors.
- However, the mean loan amounts by sector are close whether for entertainment, agriculture, food, retail, clothing, health or other stuffs. The differences in total loan amount is due to the total number of loans. 

The conclusion is that: when considering the distribution of loans by sector on a global scale (loans from all countries combined), mean loan amount does not strongly depends on sector of activities. But there are significantly more loans in agriculture, food and retail than in other sectors.


In the following, we are depicting the distribution of loan amount by sector and country.

In [ ]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

## 4.2.2 Loans distribution by sector and by country <a class="anchor" id="sector-country"></a>
[Click here to go to Table of contents](#tablecontent)

### Total loan amount

The figures below shows total loan amounts by sectors that borrowers from each country obtained as percentage of global loan mount. The tables contains the list of 10 countries with highest sector-wise loan amounts. The last row in the table is the sum of total loan amounts in these 10 countries.

The figures lead to the following conclusion: in terms of total loan amount, the distribution of loans exhibits an important geographical sector-wise localisation. 
- In other words, in a given sector of activities, the total loan amounts tend be concentrated in a handful of countries (or regions). The loans in 10 first countries commonly sum up to more than 55% of the worldwide sector-wise loan (e.g. in agriculture, clothing, construction, education...), sometimes up to 70%-80% (in sectors like food, health and entertainment).
- The countries with highest loan amount vary from one sector to another. For instance, in the sector of services and entertainment, US borrowers obtain respectively 20% and 55% of the total loan. In retail, borrowers from Philippines obtain 20% of total loan. Regarding personal uses, borrowers from Cambodia hold up to 28%. Concerning housing, borrowers from Elsalvador and Vietnam share 28% of the total loan.

In [ ]:
#%% % of each sector in each country

dfaux = dfkv.groupby(['country', 'sector']).sum()
dfaux = dfaux.unstack(level = -1)
dfaux.fillna(value = 0., inplace = True)


dfa = 100 * dfaux / dfaux.sum(axis = 0)

sectors = ['Agriculture', 'Arts', 'Clothing', 'Construction', 'Education', 'Entertainment', 'Food', 'Health', 'Housing', 'Manufacturing', 'Personal Use', 'Retail', 'Services', 'Transportation', 'Wholesale']

sectors = ['Agriculture', 'Clothing', 'Construction', 'Education', 'Entertainment', 'Food', 'Health', 'Housing',  'Services', 'Personal Use', 'Retail']


for sector in sectors:
    print('\n')
    print('Sector ' + sector + ' : Loan amount per country (% of worldwide total loan amount in this sector) ')

    dfaux = dfa.xs(sector, axis = 1, level = 1)
    dfaux.columns = [ 'Loan amount (% of worldwide total loan amount in the sector)']
    
    country_list = data_wb.index.get_level_values('country').unique().tolist()
    
    for i in country_list:
        if not(i in dfaux.index.tolist()):
            dfaux = dfaux.append( pd.DataFrame( index = [i], data = [0.0], columns = dfaux.columns ) )
    dfaux.drop(['Virgin Islands (U.S.)'], inplace = True)
    plot_world_map_from_series( dfaux )
    print(dfaux.sort_values(by =[ 'Loan amount (% of worldwide total loan amount in the sector)'], ascending = False).head(10))
    
    print(dfaux.sort_values(by =[ 'Loan amount (% of worldwide total loan amount in the sector)'], ascending = False).head(10).sum())    
    
    print('- - - - - - - - - - - - - - - - - -  - - - - - - - - - - - -  - - - - - - - - - - - -')
    print('\n')

### Distribution by relative measures: mean loan amount / GDP per capita, # loans / population size

We considered the sector-wise distribution of loans by total loan amount. This measure is important to know how loans are distributed on a global level. On a local level, as previously explained, the use of relative measures (namely mean loan amount / GDP per capita, # loans / population size) allows one to assess the effects of loans on the local population.

In the following, the relative measures are depicted on figures. The tables shows 10 countries with highest measures. Only two sectors are presented, however, by changing the list of sectors, one can easily obtain results for other sectors.

In [ ]:
#sectors = ['Agriculture', 'Arts', 'Clothing', 'Construction', 'Education', 'Food', 'Health', 'Housing', 'Manufacturing', 'Retail', 'Services', 'Transportation', 'Wholesale']
sectors = ['Agriculture',  'Food']


for sector in sectors:

    print('Sector ' + sector + ' : Distribution of loans in terms of number of loans, mean loan amount, mean amount / GDP per capita')

    # total loan amount per country per sector
    df1 = dfkv[ dfkv['sector'] == sector ].groupby(by = ['country', 'year'])['loan_amount'].sum()
    df1.name = 'Total loan amount'
    # mean loan amount per country per sector
    df2 = dfkv[ dfkv['sector'] == sector ].groupby(by = ['country', 'year'])['loan_amount'].mean()
    df2.name = 'Mean loan amount'
    
    # number of loans per country per sector
    df3 = dfkv[ dfkv['sector'] == sector ].groupby(by = ['country', 'year'])['loan_amount'].size()
    df3.name = '# loans'
    
    
    
    df = pd.concat([data_wb, df1, df2, df3 ], axis = 1, join = 'outer')

    df['# loans in 10000 inhabitants'] = 10000.*df['# loans'] /df['Total population']
    
    df['Mean loan amount / GDP per capita (current US$) (%)'] = 100.* df['Mean loan amount'] / df['GDP per capita (current US$)']
    

    # Mean loan amount /year
    dfaux = df.reset_index().groupby('country')['Mean loan amount'].mean()
    dfaux = dfaux.fillna(value = 0.)
    dfaux.drop(['Virgin Islands (U.S.)'], inplace = True)
    #plot_barh_from_series( dfaux.sort_values(ascending = False).head(10) )
    plot_world_map_from_series( dfaux)
    print( dfaux.sort_values(ascending = False).head(10) )
    print('- - - - - - - - - - - - - - - ')
    print('\n')
    
    # Mean loan amount / gdp / year
    dfaux = df.reset_index().groupby('country')['Mean loan amount / GDP per capita (current US$) (%)'].mean()
    dfaux = dfaux.fillna(value = 0)
    dfaux.drop(['Virgin Islands (U.S.)'], inplace = True)
    plot_world_map_from_series( dfaux, filename = 'Mean loan vs. gdp per capita')
    print( dfaux.sort_values(ascending = False).head(10) )
    print('- - - - - - - - - - - - - - - ')
    print('\n')
    # mean number of loans per year
    dfaux = df.reset_index().groupby('country')['# loans'].mean()
    dfaux = dfaux.fillna(value = 0)
    dfaux.drop(['Virgin Islands (U.S.)'], inplace = True)
    plot_world_map_from_series( dfaux, filename = 'Mean number of loans per year' )
    print( dfaux.sort_values(ascending = False).head(10) )
    print('- - - - - - - - - - - - - - - ')
    print('\n')
    
    # total number of loans / 10000 habitants in all years
    dfaux = df.reset_index().groupby('country')['# loans in 10000 inhabitants'].mean()
    # remove Samoa (too different)
    dfaux = dfaux[ ~dfaux.index.isin([ 'Samoa'] ) ]
    # replace nan with 0 (no loan)
    dfaux = dfaux.fillna(value = 0)
    dfaux.drop(['Virgin Islands (U.S.)'], inplace = True)
    plot_world_map_from_series( dfaux , filename = 'N# loans in 10000 inhabitants')
    print( dfaux.sort_values(ascending = False).head(10) )
    
    print('- - - - - - - - - - - - - - - - - -  - - - - - - - - - - - - - - - - - - - - - - - - ')
    print('\n')

# 4.3 Loans distribution by borrowers gender <a class="anchor" id="gender"></a>
[Click here to go to Table of contents](#tablecontent)

## 4.3.1 Worldwide scale <a class="anchor" id="gender-ww"></a>
[Click here to go to Table of contents](#tablecontent)

As one can see below, on a global scale, female borrowers represent approximately 80% of the total number and the total loan amount. 

In [ ]:
dfaux = data_kvloans[ [u'country', u'year', u'activity', u'sector', u'loan_amount', u'borrower_genders']]

In [ ]:
# replace 4221 missing values in gender by female (the most popular)
dfaux.loc[ dfaux['borrower_genders'].isnull() , 'borrower_genders']  = 'female'
dfaux.loc[ dfaux['borrower_genders'].str.contains('female'), 'borrower_genders' ] = 'female'
dfaux.loc[ ~dfaux['borrower_genders'].str.contains('female') , 'borrower_genders'] = 'male'

In [ ]:
trace = go.Pie(sort = False,labels=dfaux.groupby('borrower_genders').size().index.tolist(), values=list(dfaux.groupby('borrower_genders').size().values))
fig = {
    'data': [trace],
    'layout': {'title': '# loans by gender'}
     }

py.iplot(fig)

In [ ]:
trace = go.Pie(sort = False,labels=dfaux.groupby('borrower_genders')['loan_amount'].sum().index.tolist(), values=list(dfaux.groupby('borrower_genders')['loan_amount'].sum().values))
fig = {
    'data': [trace],
    'layout': {'title': 'Total loan amount by gender'}
     }

py.iplot(fig)

## 4.3.2 National scale <a class="anchor" id="gender-nat"></a>
[Click here to go to Table of contents](#tablecontent)

On the national level, it is worth calculating the percentage of female borrowers. 

In [ ]:
# frequency of borrowers by gender
dfaux2 = dfaux.groupby(by='country')['borrower_genders'].value_counts(normalize=True)

In [ ]:
dfaux2.head(5)

The following histogram and box plot shows that in almost all countries where there are Kiva loans, female borrowers represent at least 50% of the total number. 

In [ ]:
plt.figure()
dfaux2.loc[:,'female'].plot.hist()
plt.xlabel('Female borrower (% of total number)')
plt.show()

In [ ]:
plt.figure()
dfaux2.loc[:,'female'].plot(kind = 'box')
plt.ylabel('Female borrower (% of total number)')
plt.title('')
plt.show()

The same data presented on the world map leads to:

In [ ]:
countrylist = data_wb.reset_index().set_index('country').index.unique()
dfaux3 = pd.concat( [ pd.DataFrame(index = countrylist), dfaux2.loc[:,'female'] ], axis = 1, join = 'outer' )
dfaux3.columns = ['Female borrower (% of total number)']
plot_world_map_from_series( dfaux3)

In [ ]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

# 5. Welfare assessment <a class="anchor" id="welfare-assessment"></a>
[Click here to go to Table of contents](#tablecontent)

The objective of this section is to assess the welfare levels of borrowers given the loan details. We propose to use a clustering technique (K-Means) to evaluate the welfare levels of borrowers. The idea behind is that borrowers with similar loan details belong to the same cluster which represents a welfare level. Note that hereafter we are not going to make the judgment whether one welfare level is higher than another. This conclusion can be obtained by looking thoroughly at the clusters obtained after the analysis.

The loan details that will be utilized for clustering comprise:

- MPI of countries: which reflects the overal poverty level of the country. This is important for an assessment at the global level.
- Loan amount / GDP per capita: which accounts for both the loan amount and its relative value with respect to the borrowers's income level
- Sector & loan use: which shows the activity of the borrower
- term in months & repayment interval: which are related to the behaviour of the borrower.

Other measures might be used in the clustering process, e.g. local MPI if available.

# 5.1 Worldwide welfare assessment <a class="anchor" id="ww-assessment"></a>
[Click here to go to Table of contents](#tablecontent)

We start with assessing the welfare level of borrowers on the worldwide scale: all countries, all sectors, all loan uses ... 

## 5.1.1 Categorical variables <a class="anchor" id="ww-assessment-cat"></a>
[Click here to go to Table of contents](#tablecontent)

In [ ]:
df = data_kvloans[ [ 'loan_amount', 'activity', 'sector', 'use', 'country_code', 'country', 'region', 'term_in_months', 'borrower_genders', 'repayment_interval','year'] ]

In [ ]:
# add GDP per capita to the dataframe
df = pd.concat( [df, pd.DataFrame(columns = ['GDP per capita (current US$)'],index = df.index, data = [ np.nan ] * len(df) ) ], axis = 1, join = 'outer' )

      
for country, year in data_wb.reset_index()[[ 'country', 'year' ]].values:
    df.loc[(df['country'] == country) & (df['year'] == year), 'GDP per capita (current US$)'] = data_wb.loc[ country, year ]['GDP per capita (current US$)']


df['Loan amount / GDP per capita (%)' ]  = 100* df['loan_amount'] / df['GDP per capita (current US$)']

# MPI country
df = pd.concat( [df, pd.DataFrame(columns = ['MPI country'],index = df.index, data = [ np.nan ] * len(df) ) ], axis = 1, join = 'outer' )
for country in data_mpi[ 'Country'].unique().tolist():
    df.loc[ (df['country'] == country) , 'MPI country'] = data_mpi.loc[  (data_mpi['Country'] == country) , 'Country_MPI' ].unique()[0]

### Labelling the sectors
Categorical variables are used to represent the sectors of borrowers

In [ ]:
labels_sector = preprocessing.LabelEncoder()
labels_sector.fit( df['sector'].unique().tolist() )
df.loc[:,'sector label'] = labels_sector.transform( df['sector'] )

In [ ]:
pd.DataFrame.from_dict( { 'Sector': df['sector'].unique().tolist(), 'Sector label': list(labels_sector.transform( df['sector'].unique().tolist() ) ) }) 

### Labelling the repayment interval
Categorical variables are used to represent the repayment interval

In [ ]:
labels_repayment = preprocessing.LabelEncoder()
labels_repayment.fit( df['repayment_interval'].unique().tolist() )
df.loc[:,'repayment interval label'] = labels_repayment.transform( df['repayment_interval'] )

In [ ]:
pd.DataFrame.from_dict( { 'Repayment interval': df['repayment_interval'].unique().tolist(), 'Repayment interval label': list(labels_repayment.transform( df['repayment_interval'].unique().tolist() ) ) }) 

### Labelling the borrowers genders
Categorical variables are used to represent the gender of borrowers

In [ ]:
# replace missing values by female (the most popular)
df.loc[ df['borrower_genders'].isnull() , 'borrower_genders']  = 'female'
df.loc[ df['borrower_genders'].str.contains('female'), 'borrower_genders' ] = 'female'
df.loc[ ~df['borrower_genders'].str.contains('female') , 'borrower_genders'] = 'male'

In [ ]:
labels_gender = preprocessing.LabelEncoder()
labels_gender.fit( df['borrower_genders'].unique().tolist() )
df.loc[:,'gender label'] = labels_gender.transform( df['borrower_genders'] )

In [ ]:
pd.DataFrame.from_dict( { 'Gender': df['borrower_genders'].unique().tolist(), 'Gender label': list(labels_gender.transform( df['borrower_genders'].unique().tolist() ) ) }) 

## 5.1.2 Loan activity clustering <a class="anchor" id="ww-assessment-loan-uses-clustering"></a>
[Click here to go to Table of contents](#tablecontent)

To classify the activities of all borrowers, natural language processing technique will be used. The idea is to tokenize the activities, detect stems of words and their frequencies in the text then retrieving the important features. The Natural Language Toolkit https://www.nltk.org/ is used.

In [ ]:
# load nltk's SnowballStemmer as variabled 'stemmer'
stemmer = SnowballStemmer("english")
#print(stemmer)


# here I define a tokenizer and stemmer which returns the set of stems in the text that it is passed

def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [ ]:
#df['use'] = df['use'].str.replace('buy','purchase')
text = df['activity'].values.astype(str).tolist() # convert nan cell to string

In [ ]:
# create a dictionary of tokens and stems
totalvocab_stemmed = []
totalvocab_tokenized = []
for i in text:
    
    allwords_tokenized = tokenize_only(i)
    totalvocab_tokenized.extend(allwords_tokenized)
    
totalvocab_tokenized = list( set( totalvocab_tokenized  ) )

totalvocab_stemmed = [ stemmer.stem(t) for t in totalvocab_tokenized ]
#pickle.dump(totalvocab_stemmed, open("../input/cm-kiva-nlp/totalvocab_stemmed.p","wb"))
#totalvocab_stemmed = pickle.load(open("../input/cm-kiva-nlp/totalvocab_stemmed.p","rb"))

vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed)

#vocab_frame.info()

In [ ]:
# term frequency - inverse document frequency :  is a numerical statistic reflecting how important a word is to a document in a collection or corpus
# take into account only tokens that appear in more than 5% less than 95% of documents. Those appearing more than 95% of docs tend to be irrelevant
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, max_features= 20,\
                                 min_df=0.01, stop_words='english', norm = 'l2',\
                                 use_idf=True, analyzer = 'word', tokenizer=tokenize_and_stem, ngram_range= (1,1) )

#This operation takes around 20 minutes of computational time, therefore, I computed the matrix offline and use it as a data source
#tfidf_matrix = tfidf_vectorizer.fit_transform(text)
#pickle.dump(tfidf_vectorizer, open("../input/cm-kiva-nlp/tfidf_vectorizer.p","wb"))
#pickle.dump(tfidf_matrix, open("../input/cm-kiva-nlp/tfidf_matrix.p","wb"))

tfidf_vectorizer = pickle.load( open("../input/cm-kiva-nlp/tfidf_vectorizer.p","rb") )
tfidf_matrix = pickle.load( open("../input/cm-kiva-nlp/tfidf_matrix.p","rb") )

The important stems detected are the following:

In [ ]:
terms = tfidf_vectorizer.get_feature_names()
len(terms)
print(terms[:])

In [ ]:
#vocab_frame.loc[ terms]
#vocab_frame.loc[ 'like']

The loan activities are assigned in different clusters as a function of the tokens (words) they contain.

In [ ]:
#%% CLUSTERING by loan activity

num_clusters = 50

km = KMeans(n_clusters=num_clusters, random_state = 1)

#This operation takes several minutes of computational time, therefore, I computed the matrix offline and use it as a data source

#km.fit(tfidf_matrix)
#pickle.dump(km, open("../input/cm-kiva-nlp/km.p","wb"))

km = pickle.load( open("../input/cm-kiva-nlp/km.p","rb") )

clusters = km.labels_.tolist()

In [ ]:
df['activity label']  = clusters
#df['activity label'].value_counts()

In [ ]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

## 5.1.3 Loan activity classification <a class="anchor" id="ww-assessment-loan-uses-prediction"></a>
[Click here to go to Table of contents](#tablecontent)

Given a new loan demands (as follows), their loan activities can be classified using the obtained K-means object.

In [ ]:
df.head(2)

The predictions are exactly those expected:

In [ ]:
# prediction: assume that loanuses_predict contains the loan uses of new demands
loanact_predict = df.loc[0:1,'activity'].values.astype(str).tolist() # convert nan cell to string
tfidf_matrix_predict = tfidf_vectorizer.transform(loanact_predict)
# loan uses classes for the new demand are predicted by:
loanact_classes = km.predict( tfidf_matrix_predict )
print(loanact_classes )

Each cluster of activity is characterized by certain terms listed in the following. It is worth emphasizing that the clustering algorithm appears to reveal the borrowers' activities with great details. For instance, in the sector of Agriculture, cluster 14 contains those activities related to Fruits & Vegetables, whereas cluster 6 is contains the keyword Pigs, cluster 10: fishing, cluster 16: clothing. This will not be obtained using simply a value_counts function. All clusters are dedicated to a specific activity, except cluster 0 which contains a lot of items. Those activities are actually not as frequent as others (when considering the stems, or roots of words), therefore given the limited size of the features that we expect, they were gathered in a single cluster. It is really great, since we could reduce the activity column to a categorical variable of 20 levels instead of 163 unique values as the data show.

In [ ]:
for i in df['activity label'].unique().tolist():
    print('Activity cluster %d is characterized by:'%i)
    print(df[ df['activity label'] == i][ 'activity' ].unique().tolist() )
    
    print('\n')

In [ ]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

## 5.1.4 Clustering of loans - welfare assessment of borrowers in the database <a class="anchor" id="ww-assessment-borrowers"></a>
[Click here to go to Table of contents](#tablecontent)


The measures used for welfare assessment ares:

In [ ]:
col_list = [ 'MPI country', 'Loan amount / GDP per capita (%)', 'sector label', 'activity label', 'gender label' ,  'term_in_months', 'repayment interval label']

In [ ]:
print(col_list)

in which:
- MPI country indicates the welfare level of the borrower's country's population
- Loan amount / GDP per capita (%) might be related to the relative welfare level of the borrower in her/his country, especially when it is coupled with the loan term in months and the repayment interval. There is a certain difference between a small loan amount (with respect to the national GDP per capita) which is distributed in long period of times and a big amount distributed in few months.
- sector and activity are borrowers' characteristics

The correlation matrix shows that there is no systematic strong correlation among them (except for MPI - loan amount / GDP per capita and sector - activities ), thus assuring that none of them is redundant. It is worth noting that the important correlation between activities and sector is expected. However, details on activities are not redundant but allow us to distinguish loans in the same sector.

In [ ]:
plot_correlation_matrix( df[col_list].corr().round(2), filename = 'correlation matrix classfication' )

The clustering process can now be applied.

In [ ]:
# fill missing values with median
fill_missing_values = preprocessing.Imputer(strategy = 'median').fit( df[ col_list ]  )

fill_missing_values.transform( df[ col_list ]  )


num_loan_clusters = 50

loan_clusters = KMeans(n_clusters=num_loan_clusters , random_state = 1)

#loan_clusters.fit( fill_missing_values.transform( df[ col_list ]  ) )
#pickle.dump(loan_clusters, open("../input/cm-kiva-nlp/loan_clusters.p","wb"))

loan_clusters = pickle.load( open("../input/cm-kiva-nlp/loan_clusters.p","rb") )

clusters = loan_clusters.labels_.tolist()

df['Loan class'] = clusters

For each loan class, the statistics of all details can be obtained in the following (hereafter, only some largest clusters are presented):

In [ ]:
#%% plot loans in each loan use group

#cl_list = df['Loan class'].unique().tolist()
# plot 3 largest loan classes
cl_list = df['Loan class'].value_counts().index[0:3].tolist()

for i in cl_list :
    print('Loan class %d'%i)
    print( df[ df['Loan class'] == i][ col_list ].describe() )
    print('- - - - - - - - - - - - - - - - - -  - - - - - - - - - - - - ')
    print('\n')


Let's compare the difference between two welfare levels represented by five biggest clusters. One can observe the significant differences between them in terms of MPI, loan amount / GDP per capita and sector.

In [ ]:
for y in [ 'MPI country', 'Loan amount / GDP per capita (%)', 'sector label', 'activity label' ,'term_in_months']:
    plt.figure()
    sns.boxplot(x="Loan class", y= y,  data=df[ df['Loan class'].isin(df['Loan class'].value_counts().index[0:5].tolist())]);

In terms of repayment interval, there exist a clear difference between clusters, with monthly payment being more or less frequent than irregular payment, both being dominant with respect to two other intervals.

In [ ]:
plt.figure()
sns.countplot(y="Loan class", hue = 'repayment_interval',  data=df[ df['Loan class'].isin(df['Loan class'].value_counts().index[0:5].tolist())]);

In terms of borrowers genders, the trend is similar throughout different clusters with women being in large number compared to men.

In [ ]:
plt.figure()
sns.countplot(y="Loan class", hue = 'gender label',  data=df[ df['Loan class'].isin(df['Loan class'].value_counts().index[0:5].tolist())]);

In [ ]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

## 5.1.5 Welfare assessment for new borrowers<a class="anchor" id="ww-assessment-new-borrowers"></a>
[Click here to go to Table of contents](#tablecontent)

Given new borrowers as with loan details as follows:

In [ ]:
df.loc[ 0:1, : ]

one can predict the welfare levels of these borrowers using the K-means object computed above. Note that, first of all, the classification of loan uses, sectors of activity, repayment behaviour, ... has to be conducted. The loan classes predicted are exactly those in the data:

In [ ]:
loan_classes_predict = loan_clusters.predict( fill_missing_values.transform( df.loc[ 0:1, col_list ]  ) )
print(loan_classes_predict)

The descriptive statistics of the classes in the tables presented below allow us to assess the groups to which each new loan demand shall be classified.

For instance, the first loan demand belongs to a group with the mean loan amount / GDP per capita equalling 18.45%, whereas the second demand goes to a group of mean value 48.97%. These predictions are not that far from the actual numbers in the data.

In [ ]:
for i in loan_classes_predict:
    print('Loan class %d'%i)
    print(df[ df['Loan class']== i][ col_list ].describe())
    print('\n')

In [ ]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

## 5.2 Nation-wise welfare assessment <a class="anchor" id="nw-assessment"></a>
[Click here to go to Table of contents](#tablecontent)

The above classification-based approach for assessing welfare levels of borrowers can be utilized at a given country. For the sake of simplicity, clustering functions are defined in order to conduct analysis on as given country in a simple manner. Hereafter, welfare assessment on some specific countries is presented. Note that for a given country, it would be ideal to use regional MPI determined at borrower's location. Although the same GDP per capita is used for borrowers from the same country, the ration loan amount / GDP per capita is still used for clustering. This allows to account for the importance of loan amount with respect to the national income level.

In [ ]:
def kmeans_func(df, nclusters = 10):
    fill_missing_values = preprocessing.Imputer(strategy = 'median').fit( df )
    
    clusters = KMeans(n_clusters=num_loan_clusters , random_state = 1)
    clusters.fit( preprocessing.Imputer(strategy = 'median').fit_transform( fill_missing_values.transform(df) ))
    
    return clusters.labels_.tolist()

In [ ]:
# IDEAL CASE; MPI OF REGIONS should be included.
col_list = [ 'Loan amount / GDP per capita (%)', 'sector label', 'activity label', 'gender label',  'term_in_months', 'repayment interval label']

### Samoa

In [ ]:
country = 'Samoa'

dfaux =  df.loc[ df['country'] == country ]
dfaux = dfaux.drop(['Loan class'], axis = 1)

cl_predict = kmeans_func( dfaux.loc[:,col_list] , nclusters = 5 )
dfaux = pd.concat([dfaux, pd.DataFrame(data= cl_predict, index = dfaux.index, columns = ['Loan class'] ) ], axis = 1, join = 'outer' )

In [ ]:
#%% plot loans in each loan use group

#cl_list = dfaux['Loan class'].unique().tolist()
cl_list = dfaux['Loan class'].value_counts().index[0:3].tolist()

#%%
for i in cl_list :
    print('Loan class %d'%i)
    print(dfaux[ dfaux['Loan class'] == i][ col_list ].describe())
    print('- - - - - - - - - - - - - - - - - -  - - - - - - - - - - - - ')
    print('\n')
    

The differences between clusters are depicted below, which are very pronounced for the loan amount.

In [ ]:
for y in [ 'Loan amount / GDP per capita (%)', 'sector label', 'activity label' ,  'term_in_months', 'repayment interval label']:
    plt.figure()
    sns.boxplot(x="Loan class", y= y,  data=dfaux[ dfaux['Loan class'].isin(dfaux['Loan class'].value_counts().index[0:5].tolist())]);
    
plt.figure()
sns.countplot(y="Loan class", hue = 'gender label',  data=dfaux[ dfaux['Loan class'].isin(dfaux['Loan class'].value_counts().index[0:5].tolist())]);

In [ ]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

### Philippines

In [ ]:
country = 'Philippines'

dfaux =  df.loc[ df['country'] == country ]
dfaux = dfaux.drop(['Loan class'], axis = 1)

cl_predict = kmeans_func( dfaux.loc[:,col_list] , nclusters = 5 )
dfaux = pd.concat([dfaux, pd.DataFrame(data= cl_predict, index = dfaux.index, columns = ['Loan class'] ) ], axis = 1, join = 'outer' )

cl_list = dfaux['Loan class'].value_counts().index[0:3].tolist()

#%%
for i in cl_list :
    print('Loan class %d'%i)
    print(dfaux[ dfaux['Loan class'] == i][ col_list ].describe())
    print('- - - - - - - - - - - - - - - - - -  - - - - - - - - - - - - ')
    print('\n')
    
for y in [ 'Loan amount / GDP per capita (%)', 'sector label', 'activity label' ,  'term_in_months', 'repayment interval label']:
    plt.figure()
    sns.boxplot(x="Loan class", y= y,  data=dfaux[ dfaux['Loan class'].isin(dfaux['Loan class'].value_counts().index[0:5].tolist())]);
    
plt.figure()
sns.countplot(y="Loan class", hue = 'gender label',  data=dfaux[ dfaux['Loan class'].isin(dfaux['Loan class'].value_counts().index[0:5].tolist())]);

In [ ]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

### Kenya

In [ ]:
country = 'Kenya'

dfaux =  df.loc[ df['country'] == country ]
dfaux = dfaux.drop(['Loan class'], axis = 1)

cl_predict = kmeans_func( dfaux.loc[:,col_list] , nclusters = 5 )
dfaux = pd.concat([dfaux, pd.DataFrame(data= cl_predict, index = dfaux.index, columns = ['Loan class'] ) ], axis = 1, join = 'outer' )

cl_list = dfaux['Loan class'].value_counts().index[0:3].tolist()

#%%
for i in cl_list :
    print('Loan class %d'%i)
    print(dfaux[ dfaux['Loan class'] == i][ col_list ].describe())
    print('- - - - - - - - - - - - - - - - - -  - - - - - - - - - - - - ')
    print('\n')
    
for y in [ 'Loan amount / GDP per capita (%)', 'sector label', 'activity label' ,  'term_in_months', 'repayment interval label']:
    plt.figure()
    sns.boxplot(x="Loan class", y= y,  data=dfaux[ dfaux['Loan class'].isin(dfaux['Loan class'].value_counts().index[0:5].tolist())]);
    
plt.figure()
sns.countplot(y="Loan class", hue = 'gender label',  data=dfaux[ dfaux['Loan class'].isin(dfaux['Loan class'].value_counts().index[0:5].tolist())]);

In [ ]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

## 5.3 Welfare assessment by country and sector <a class="anchor" id="ns-assessment"></a>
[Click here to go to Table of contents](#tablecontent)

In a given country and sector, welfare levels of borrowers can also be easily determined. We consider some cases:

In [ ]:
# IDEAL CASE; MPI OF REGIONS should be included.
col_list = [ 'Loan amount / GDP per capita (%)', 'activity label', 'gender label',  'term_in_months', 'repayment interval label']


### India - Agriculture
When detailed loan uses are taken into account

In [ ]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

In [ ]:
country = 'India'
sector = 'Agriculture'

dfaux =  df.loc[ (df['country'] == country ) & (df['sector'] == sector) ]
dfaux = dfaux.drop(['Loan class'], axis = 1)

cl_predict = kmeans_func( dfaux.loc[:,col_list] , nclusters = 5 )
dfaux = pd.concat([dfaux, pd.DataFrame(data= cl_predict, index = dfaux.index, columns = ['Loan class'] ) ], axis = 1, join = 'outer' )

cl_list = dfaux['Loan class'].value_counts().index[0:3].tolist()

#%%
for i in cl_list :
    print('Loan class %d'%i)
    print(dfaux[ dfaux['Loan class'] == i][ col_list ].describe())
    print('- - - - - - - - - - - - - - - - - -  - - - - - - - - - - - - ')
    print('\n')
    
    
for y in [ 'Loan amount / GDP per capita (%)' ,  'term_in_months', 'repayment interval label']:
    plt.figure()
    sns.boxplot(x="Loan class", y= y,  data=dfaux[ dfaux['Loan class'].isin(dfaux['Loan class'].value_counts().index[0:5].tolist())]);

plt.figure()
sns.countplot(y="Loan class", hue = 'activity label',  data=dfaux[ dfaux['Loan class'].isin(dfaux['Loan class'].value_counts().index[0:5].tolist())]);

plt.figure()
sns.countplot(y="Loan class", hue = 'gender label',  data=dfaux[ dfaux['Loan class'].isin(dfaux['Loan class'].value_counts().index[0:5].tolist())]);

### India - Agriculture - Loan use

In [ ]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

When borrowers are from the same country and the same sector (e.g. Indian farmers), the detailed loan uses might give a hint of the welfare levels of borrowers. Thus, natural language processing techniques are used again for the clustering of detailed loan uses.

In [ ]:
dfaux =  df.loc[ (df['country'] == country ) & (df['sector'] == sector) ]
dfaux = dfaux.drop(['Loan class'], axis = 1)

In [ ]:
# load nltk's SnowballStemmer as variabled 'stemmer'
stemmer = SnowballStemmer("english")
#print(stemmer)


dfaux['use'] = dfaux['use'].str.replace('buy','purchase')
text = dfaux['use'].values.astype(str).tolist() # convert nan cell to string

# create a dictionary of tokens and stems
totalvocab_stemmed = []
totalvocab_tokenized = []
for i in text:
    
    allwords_tokenized = tokenize_only(i)
    totalvocab_tokenized.extend(allwords_tokenized)
    
totalvocab_tokenized = list( set( totalvocab_tokenized  ) )

totalvocab_stemmed = [ stemmer.stem(t) for t in totalvocab_tokenized ]

vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed)

#vocab_frame.info()

# term frequency - inverse document frequency :  is a numerical statistic reflecting how important a word is to a document in a collection or corpus
# take into account only tokens that appear in more than 5% less than 95% of documents. Those appearing more than 95% of docs tend to be irrelevant
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, max_features= 20,\
                                 min_df=0.01, stop_words='english', norm = 'l2',\
                                 use_idf=True, analyzer = 'word', tokenizer=tokenize_and_stem, ngram_range= (1,1) )

#This operation takes minutes of computational time, therefore, I computed the matrix offline and use it as a data source
tfidf_matrix = tfidf_vectorizer.fit_transform(text)

terms = tfidf_vectorizer.get_feature_names()
#len(terms)
#print(terms[:])

#%% CLUSTERING by loan use

num_clusters = 20

km = KMeans(n_clusters=num_clusters, random_state = 1)

#This operation takes several minutes of computational time, therefore, I computed the matrix offline and use it as a data source

km.fit(tfidf_matrix)


clusters = km.labels_.tolist()

dfaux['loan use label']  = clusters


In [ ]:
# IDEAL CASE; MPI OF REGIONS should be included.
col_list = [ 'Loan amount / GDP per capita (%)', 'activity label', 'gender label', 'loan use label',  'term_in_months', 'repayment interval label']

cl_predict = kmeans_func( dfaux.loc[:,col_list] , nclusters = 5 )
dfaux = pd.concat([dfaux, pd.DataFrame(data= cl_predict, index = dfaux.index, columns = ['Loan class'] ) ], axis = 1, join = 'outer' )

cl_list = dfaux['Loan class'].value_counts().index[0:3].tolist()

#%%
for i in cl_list :
    print('Loan class %d'%i)
    print(dfaux[ dfaux['Loan class'] == i][ col_list ].describe())
    print('- - - - - - - - - - - - - - - - - -  - - - - - - - - - - - - ')
    print('\n')
    
    
for y in [ 'Loan amount / GDP per capita (%)' ,  'term_in_months', 'repayment interval label']:
    plt.figure()
    sns.boxplot(x="Loan class", y= y,  data=dfaux[ dfaux['Loan class'].isin(dfaux['Loan class'].value_counts().index[0:5].tolist())]);

plt.figure()
sns.countplot(y="Loan class", hue = 'activity label',  data=dfaux[ dfaux['Loan class'].isin(dfaux['Loan class'].value_counts().index[0:5].tolist())]);

plt.figure()
sns.countplot(y="Loan class", hue = 'loan use label',  data=dfaux[ dfaux['Loan class'].isin(dfaux['Loan class'].value_counts().index[0:5].tolist())]);


plt.figure()
sns.countplot(y="Loan class", hue = 'gender label',  data=dfaux[ dfaux['Loan class'].isin(dfaux['Loan class'].value_counts().index[0:5].tolist())]);

One can see that the different welfare levels (represented by loan classes) are characterized by the loan amount/GDP per capita and loan uses. For instance, loan class 2 has the most loan uses belonging to loan use class 7, which is characterized by the features:

In [ ]:
ind = dfaux['loan use label'] == 7
textpr = dfaux.loc[ind, 'use'].values.astype(str).tolist() # convert nan cell to string
tfidf_vectorizer.fit(textpr)
#print(tfidf_vectorizer.get_feature_names())
# get the common terms with the collection of terms from all documents
imp_terms = vocab_frame.loc[ set(terms) & set( tfidf_vectorizer.get_feature_names() )]
imp_terms.reset_index().drop_duplicates(subset='index', keep='first', inplace=False).set_index('index')

where loan class 13 has the most loan uses belonging to loan use class 10, which contains the features:

In [ ]:
ind = dfaux['loan use label'] == 10
textpr = dfaux.loc[ind, 'use'].values.astype(str).tolist() # convert nan cell to string
tfidf_vectorizer.fit(textpr)
#print(tfidf_vectorizer.get_feature_names())
# get the common terms with the collection of terms from all documents
imp_terms = vocab_frame.loc[ set(terms) & set( tfidf_vectorizer.get_feature_names() )]
imp_terms.reset_index().drop_duplicates(subset='index', keep='first', inplace=False).set_index('index')

For loan class 23, the loan uses are characterized by:

In [ ]:
ind = dfaux['loan use label'] == 18
textpr = dfaux.loc[ind, 'use'].values.astype(str).tolist() # convert nan cell to string
tfidf_vectorizer.fit(textpr)
#print(tfidf_vectorizer.get_feature_names())
# get the common terms with the collection of terms from all documents
imp_terms = vocab_frame.loc[ set(terms) & set( tfidf_vectorizer.get_feature_names() )]
imp_terms.reset_index().drop_duplicates(subset='index', keep='first', inplace=False).set_index('index')

In [ ]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

# 6. Conclusions <a class="anchor" id="conclusions"></a>
[Click here to go to Table of contents](#tablecontent)

In the submission, we investigated the __localisation__ of loans by:

- country
- sector
- gender

using:

- absolute indicators
- __relative measures__ 

It was shown that the absolute or relative measures reveal facts on two different levels: global (worldwide) or local scales. If a comparison is of interest, e.g. a inter-countries comparision of number of loans, we strongly recommend the use of relative measures.

We propose a clustering approach for __assessing welfare levels__ of borrowers in the database, which allows a __welfare prediction__ of new loan demands. This approach made use of:

- natural language processing technique for detecting most important features in this record and clustering loan activities and uses
- k-means clustering to obtain different groups of loans, which are associated with different welfare levels
- the use of two above techniques for assessing welfare levels of new borrowers

To facilitate the use of the notebook, we provided __python functions__ that allow easy computation of figures and the proposed clustering-based welfare assessment approach. It commonly suffices to replace the name of the country, or the name of the sector in order to export figures that interest the users.